In [1]:
import arcpy
from arcpy import env
import os
import numpy as np
from arcgis import GIS
from arcgis.features import GeoAccessor
from arcgis.features import GeoSeriesAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.pivot_table(df, values='a', index='b', columns='c', aggfunc='sum', fill_value=0)
# pd.DataFrame.spatial.from_featureclass(???)  
# df.spatial.to_featureclass(location=???,sanitize_columns=False)  

# gsa = arcgis.features.GeoSeriesAccessor(df['SHAPE'])  
# df['AREA'] = gsa.area  # KNOW YOUR UNITS

In [2]:
# fill NA values in Spatially enabled dataframes (ignores SHAPE column)
def fill_na_sedf(df_with_shape_column, fill_value=0):
    if 'SHAPE' in list(df_with_shape_column.columns):
        df = df_with_shape_column.copy()
        shape_column = df['SHAPE'].copy()
        del df['SHAPE']
        return df.fillna(fill_value).merge(shape_column,left_index=True, right_index=True, how='inner')
    else:
        raise Exception("Dataframe does not include 'SHAPE' column")

In [3]:
outputs = ['.\\Outputs', "scratch.gdb", 'results.gdb']

if not os.path.exists(outputs[0]):
    os.makedirs(outputs[0])

gdb = os.path.join(outputs[0], outputs[1])
gdb2 = os.path.join(outputs[0], outputs[2])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

if not arcpy.Exists(gdb2):
    arcpy.CreateFileGDB_management(outputs[0], outputs[2])

In [4]:
segments = pd.DataFrame.spatial.from_featureclass(r"E:\Projects\ATO-Impact-Tool\helper_scripts\WFv901_Segments.shp")
summary_file = pd.read_csv(r"E:\Projects\ATO-Impact-Tool\helper_scripts\v902_SE23_Net23_Summary_SEGID.csv")

In [5]:
segments.head(3)

,FID,SEGID,BMP,EMP,DISTANCE,CO_FIPS,ROUTE,SUBAREAID,PLANAREA,AADTSTN,AADT2022,AADT2021,AADT2020,AADT2019,AADT2018,AADT2017,AADT2016,AADT2015,AADT2014,AADT2013,AADT2012,AADT2011,AADT2010,AADT2009,AADT2008,AADT2007,AADT2006,AADT2005,AADT2004,AADT2003,AADT2002,AADT2001,AADT2000,AADT1999,AADT1998,AADT1997,AADT1996,AADT1995,AADT1994,AADT1993,AADT1992,AADT1991,AADT1990,AADT1989,AADT1988,AADT1987,AADT1986,AADT1985,AADT1984,AADT1983,AADT1982,AADT1981,SUTRK2022,CUTRK2022,SUTRK2021,CUTRK2021,SUTRK2020,CUTRK2020,SUTRK2019,CUTRK2019,SUTRK2018,CUTRK2018,SUTRK2017,CUTRK2017,SUTRK2016,CUTRK2016,SUTRK2015,CUTRK2015,SUTRK2014,CUTRK2014,SUTRK2013,CUTRK2013,SUTRK2012,CUTRK2012,SUTRK2011,CUTRK2011,SUTRK2010,CUTRK2010,CCSGROUP19,FACMANADJ,FAC_APR,FAC_AUG,FAC_DEC,FAC_FAL,FAC_FEB,FAC_FRI,FAC_JAN,FAC_JUL,FAC_JUN,FAC_MAR,FAC_MAX,FAC_MAXMO,FAC_MAY,FAC_MON,FAC_NOV,FAC_OCT,FAC_SAT,FAC_SEP,FAC_SPR,FAC_SUM,FAC_SUN,FAC_THU,FAC_TUE,FAC_WDAVG,FAC_WEAVG,FAC_WED,FAC_WEMAX,FAC_WIN,SHAPE
0,0,0006_141.0,141.035,146.868,5.843301,49,0006,1,MAG,049-0005,1741,1760,1647,1517,1443,1458,1453,1385,1300,1260,1345,1360,1315,1335,1215,1315,1330,1310,1340,1365,1450,1400,1345,1335,1220,1185,1170,1140,1015,930,920,900,845,825,830,890,1060,1340,1240,965,1000,975,0.039852,0.072107,0.039773,0.072159,0.037009,0.069247,0.034769,0.062952,0.033032697,0.057406449,0.028306777,0.045450138,0.086894,0.065617,0.079450392,0.063984355,0.073246409,0.063092907,0.069628124,0.060300209,0.078625,0.066644806,0.076659734,0.063559876,0.090717089,0.052861231,COP,0,1.0366,1.0897,0.8822,1.0316,0.8504,1.1505,0.8159,1.0796,1.1047,0.9396,1.1065,5,1.1065,0.9506,0.9511,1.0684,1.0649,1.0752,1.0276,1.0914,0.8819,1.012,0.9628,0.984,0.9734,0.9773,1.0649,0.8495,"{""paths"": [[[405824.11000000034, 4423860.33], ..."
1,1,0006_146.9,146.868,149.902,3.039023,49,0006,1,MAG,049-0005,1741,1760,1647,1517,1443,1458,1453,1385,1300,1260,1345,1360,1315,1335,1215,1315,1330,1310,1340,1365,1450,1400,1345,1335,1220,1185,1170,1140,1015,930,920,900,845,825,830,890,1060,1340,1240,965,1000,975,0.039852,0.072107,0.039773,0.072159,0.037009,0.069247,0.034769,0.062952,0.033032697,0.057406449,0.028306777,0.045450138,0.086894,0.065617,0.079450392,0.063984355,0.073246409,0.063092907,0.069628124,0.060300209,0.078625,0.066644806,0.076659734,0.063559876,0.090717089,0.052861231,COP,0,1.0366,1.0897,0.8822,1.0316,0.8504,1.1505,0.8159,1.0796,1.1047,0.9396,1.1065,5,1.1065,0.9506,0.9511,1.0684,1.0649,1.0752,1.0276,1.0914,0.8819,1.012,0.9628,0.984,0.9734,0.9773,1.0649,0.8495,"{""paths"": [[[413442.55030000024, 4422753.7282]..."
2,2,0006_149.9,149.902,150.58,0.677983,49,0006,1,MAG,049-0010,2803,2834,2651,2441,2320,2610,2602,2480,2330,2250,2405,1985,1925,1950,1855,2005,2025,1315,1345,1365,2000,1930,1855,1835,1675,1630,1605,1860,1655,1510,1490,1310,1230,1195,1205,1295,1435,1575,1505,1225,1225,1200,0.096974,0.09871,0.097036,0.0988,0.095553,0.097279,0.094433,0.094132,0.0935649,0.091359145,0.09120194,0.085380989,0.120496,0.095465,0.116773747,0.094648098,0.113671756,0.094202374,0.111862613,0.092806024,0.116361,0.095978323,0.115378418,0.094435858,0.122407096,0.089086536,COP,0,1.0366,1.0897,0.8822,1.0316,0.8504,1.1505,0.8159,1.0796,1.1047,0.9396,1.1065,5,1.1065,0.9506,0.9511,1.0684,1.0649,1.0752,1.0276,1.0914,0.8819,1.012,0.9628,0.984,0.9734,0.9773,1.0649,0.8495,"{""paths"": [[[418330.7999999998, 4422866], [418..."


In [6]:
summary_file.head(3)

,SEGIDIDX,SEGID,SUBAREAID,CO_FIPS,AREATYPE,ATYPENAME,LINKS,DIST_1WY,LANES,FT,FTCLASS,CAP1HL,AM_VOL,MD_VOL,PM_VOL,EV_VOL,DY_VOL,DY_LT,DY_MD,DY_HV,FF_SPD,AM_SPD,MD_SPD,PM_SPD,EV_SPD,DY_SPD,D1_LINKS,D1_DIST,D1_LANES,D1_FT,D1_FTCLASS,D1_CAP1HL,D1_AM_VOL,D1_MD_VOL,D1_PM_VOL,D1_EV_VOL,D1_DY_VOL,D1_DY_LT,D1_DY_MD,D1_DY_HV,D1_FF_SPD,D1_AM_SPD,D1_MD_SPD,D1_PM_SPD,D1_EV_SPD,D1_DY_SPD,D2_LINKS,D2_DIST,D2_LANES,D2_FT,D2_FTCLASS,D2_CAP1HL,D2_AM_VOL,D2_MD_VOL,D2_PM_VOL,D2_EV_VOL,D2_DY_VOL,D2_DY_LT,D2_DY_MD,D2_DY_HV,D2_FF_SPD,D2_AM_SPD,D2_MD_SPD,D2_PM_SPD,D2_EV_SPD,D2_DY_SPD
0,1,0006_141.0,1.0,49.0,1.0,Rural,14,5.415,2.0,2.0,Principal Arterial,1065,358.0,512.0,417.0,381.0,1668.0,0.0,156.0,132.0,58.0,58.0,58.0,58.0,58.0,58.0,7,5.415,1.0,2.0,Principal Arterial,1065,208.0,276.9,204.2,197.7,886.8,0.0,104.1,75.3,58.0,58.0,58.0,58.0,58.0,58.0,7,5.415,1.0,2.0,Principal Arterial,1065,150.0,235.1,212.8,183.3,781.2,0.0,51.9,56.7,58.0,58.0,58.0,58.0,58.0,58.0
1,2,0006_146.9,1.0,49.0,1.0,Rural,6,3.374,2.0,2.0,Principal Arterial,1065,360.0,515.0,419.0,383.0,1677.0,1.0,156.0,132.0,58.0,58.0,58.0,58.0,58.0,58.0,3,3.374,1.0,2.0,Principal Arterial,1065,230.0,293.0,201.0,203.0,927.0,0.0,123.0,82.0,58.0,58.0,58.0,58.0,58.0,58.0,3,3.374,1.0,2.0,Principal Arterial,1065,130.0,222.0,218.0,180.0,750.0,1.0,33.0,50.0,58.0,58.0,58.0,58.0,58.0,58.0
2,3,0006_149.9,1.0,49.0,1.0,Rural,2,0.621,2.0,2.0,Principal Arterial,1065,298.2,518.8,606.7,375.0,1798.7,175.9,177.0,106.1,58.0,58.0,58.0,58.0,58.0,58.0,1,0.621,1.0,2.0,Principal Arterial,1065,200.9,299.5,284.0,202.0,986.4,85.0,117.0,59.0,58.0,58.0,58.0,58.0,58.0,58.0,1,0.621,1.0,2.0,Principal Arterial,1065,97.3,219.3,322.7,173.0,812.3,90.9,60.0,47.1,58.0,58.0,58.0,58.0,58.0,58.0
